# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


we will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, Tag  
import re
import unicodedata
import pandas as pd

and we will provide some helper functions to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, we will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
response = requests.get(static_url)
# assign the response to a object
if response.status_code == 200:
    response_object = response
    print("HTTP GET request succesfull")
else:
    print("failed to retrive data from url. response code:",response.status_code)

HTTP GET request succesfull


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup


if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    print("Beautifulsoup object created succesfully")
else:
    print("failed to create beautiful soup object")

Beautifulsoup object created succesfully


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
page_title = soup.title.text

print("page title:",page_title)

page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
html_tables = soup.find_all('table')
# Assign the result to a list called `html_tables`


Starting from the third table is our target table contains the actual launch records.


we should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [14]:
# Identify the third table (indexing starts from 0)
target_table = html_tables[2]

# Find all the <th> elements within the target table
column_headers = target_table.find_all('th')

# Extract column names using extract_column_from_header() function
column_names = [extract_column_from_header(th) for th in column_headers if extract_column_from_header(th)]

# Print the extracted column names
print("Extracted Column Names:")
for name in column_names:
    print(name)



Extracted Column Names:
Flight No.
Date and time ( )
Launch site
Payload
Payload mass
Orbit
Customer
Launch outcome


Check the extracted column names


In [15]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [16]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [22]:
extracted_row = 0
# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table row 
    for rows in table.find_all("tr"):
        # Check to see if the first table heading is as a number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # Get table element 
        row = rows.find_all('td')
        # If it is a number, save cells in a dictionary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time value
            # Append the date and time into launch_dict with keys `Date` and `Time`
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
              
            # Booster version
            # Append the booster version into launch_dict with key `Version Booster`
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print(f"- Booster version: {bv}")
            
            # Launch Site
            # Append the launch site into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(f"- Launch site: {launch_site}")
            
            # Payload
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(f"- Payload: {payload}")
            
            # Payload Mass
            # Append the payload mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(f"- Payload Mass: {payload_mass}")
            
            # Orbit
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(f"- Orbit: {orbit}")
            
            # Customer
            # Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            print(f"- Customer: {customer}")
            
            # Launch outcome
            # Append the launch outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(f"- Launch Outcome: {launch_outcome}")
            
            # Booster landing
            # Append the booster landing into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(f"- Booster Landing: {booster_landing}")

            

- Booster version: F9 v1.0B0003.1
- Launch site: CCAFS
- Payload: Dragon Spacecraft Qualification Unit
- Payload Mass: 0
- Orbit: LEO
- Customer: SpaceX
- Launch Outcome: Success

- Booster Landing: Failure
- Booster version: F9 v1.0B0004.1
- Launch site: CCAFS
- Payload: Dragon
- Payload Mass: 0
- Orbit: LEO
- Customer: NASA
- Launch Outcome: Success
- Booster Landing: Failure
- Booster version: F9 v1.0B0005.1
- Launch site: CCAFS
- Payload: Dragon
- Payload Mass: 525 kg
- Orbit: LEO
- Customer: NASA
- Launch Outcome: Success
- Booster Landing: No attempt

- Booster version: F9 v1.0B0006.1
- Launch site: CCAFS
- Payload: SpaceX CRS-1
- Payload Mass: 4,700 kg
- Orbit: LEO
- Customer: NASA
- Launch Outcome: Success

- Booster Landing: No attempt
- Booster version: F9 v1.0B0007.1
- Launch site: CCAFS
- Payload: SpaceX CRS-2
- Payload Mass: 4,877 kg
- Orbit: LEO
- Customer: NASA
- Launch Outcome: Success

- Booster Landing: No attempt

- Booster version: F9 v1.1B1003
- Launch site: VAFB
-

After we have fill in the parsed launch record values into `launch_dict`, we can create a dataframe from it.


In [25]:
import pandas as pd

# Ensure all lists in launch_dict have the same length
max_length = max(len(value) for value in launch_dict.values())
for key in launch_dict:
    launch_dict[key] += [None] * (max_length - len(launch_dict[key]))

# Convert the dictionary to a DataFrame
launch_df = pd.DataFrame.from_dict(launch_dict)

# Print the DataFrame to verify
print(launch_df)

    Flight No. Launch site                               Payload Payload mass  \
0            1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1            1       CCAFS                                Dragon            0   
2            2       CCAFS                                Dragon       525 kg   
3            3       CCAFS                          SpaceX CRS-1     4,700 kg   
4            4       CCAFS                          SpaceX CRS-2     4,877 kg   
..         ...         ...                                   ...          ...   
117        117         KSC                              Starlink   ~14,000 kg   
118        118       CCSFS                              Starlink    15,600 kg   
119        119         KSC                         SpaceX CRS-22     3,328 kg   
120        120       CCSFS                                 SXM-8     7,000 kg   
121        121        None                                  None         None   

    Orbit   Customer Launch

In [26]:
# Convert the dictionary to a DataFrame
launch_df = pd.DataFrame.from_dict(launch_dict)

# Print the DataFrame to verify
print(launch_df)

    Flight No. Launch site                               Payload Payload mass  \
0            1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1            1       CCAFS                                Dragon            0   
2            2       CCAFS                                Dragon       525 kg   
3            3       CCAFS                          SpaceX CRS-1     4,700 kg   
4            4       CCAFS                          SpaceX CRS-2     4,877 kg   
..         ...         ...                                   ...          ...   
117        117         KSC                              Starlink   ~14,000 kg   
118        118       CCSFS                              Starlink    15,600 kg   
119        119         KSC                         SpaceX CRS-22     3,328 kg   
120        120       CCSFS                                 SXM-8     7,000 kg   
121        121        None                                  None         None   

    Orbit   Customer Launch

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [27]:
# Export the DataFrame to a CSV file
launch_df.to_csv('launch_data.csv', index=False)

print("CSV file has been successfully created.")

CSV file has been successfully created.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
